### Latex Macros
$\newcommand{\Re}[1]{{\mathbb{R}^{{#1}}}}
\newcommand{\Rez}{{\mathbb{R}}}$

# Basics of link prediction
As we discussed last time, recommender systems can be viewed as an exercise in link prediction. 
From simplest to more complex, the types of graphs that come into  play are: 
1) Undirected or directed graphs
2) Bipartite Graphs
3) Heterogeneous graphs
    * Heterogeneous nodes and edges
4) Temporal graphs
    * homogeneous
    * heterogeneous
    * multi-graphs
    * hypergraphs

# Link-prediction algorithms
Hamilton (2020)](https://www.cs.mcgill.ca/~wlh/grl_book/), considers the following broad approaches
1. Node similarity: <br/>
    Each node is assigned a similarity value (a number $S\in\Rez$). This can be used as a node features. 
2. A similarity $S(u,v)$ is defined between nodes $u,v$. Rank potential links $(u,v)$ according to $S(u,v)$. <br/>

## Examples of similarity metrics: 
### Neighborhood overlap measures
* Sorenson overlap
* Salton overlap
* Jaccard overlap: $$ \frac{|\cal{N}(u) \cap \cal{N}(v)|}{|\cal{N}(u) \cup \cal{N}(v)|}$$
* Reource Allocation (RA) index: 
$$  S_{RA}(v_1, v_2) = \sum_{u\in\cal{N}(v_1)\cap\cal{N}(v_2)} \frac{1}{d_u} $$
* Adamic-Adar (AA) index: 
$$  S_{AA}(v_1,v_2) = \sum_{u\in\cal{N}(v_1)\cap\cal{N}(v_2)} \frac{1}{\log(d_u)} $$

### Global overlap measures
* Katz index
$$ S_{Katz}(u,v) = \sum_{i=1}^\infty \beta_i A^i(u,v) $$ with $\beta\in\Rez$. 
* Normalized Katz index:
$$
S_{LNH}(u,v) = I(u,v) + \frac{2m}{d_u d_v} \sum_{i=0}^\infty \beta^i\lambda_1^(1-i} A^i(u,v) 
$$ 
with the analytical simplification: 
$$
 S_{LNH}(u,v) = 2 \alpha m \lambda_1 D^{-1} (I - \frac{\beta}{\lambda_1} A)^{-1} D^{-1},
$$
where
$I \in \Re{|V|\times|V|}$ identity matrix, $\lambda_1$ is maximum eigenvalue of $A$. 

### Random Walk measures
They are reminiscent of the algorithm word2vec. Start from two nodes and execute pairs of random walks from nodes $u$ and $j$ based on the 
transition matrix $P=AD^{-1}$. Measure the common nodes between the random walks. 

More deterministic, solve for the steady state of the random walk, defined as $q_u$. <br/>
$q_u(v)$ is the probability that a path starting at node $u$ ends at node $v$  Define a random walk similarity measure as
$$
S_{RW}(u,v) = q_u(v)+ q_v(u)
$$

There are many more metrics that can and are used. 

## Using similarity metrics for link preduction
Given a graph $G(V,E)$, consider the existence of links on a set of edges $e_{i} \in \cal{E}$. <br/>
Compute $S(e_i[0], e_i[1])$ for all edges in $\cal{E}$ and rank the edges. Top $n$ edges are the most likely new links. 

## Disadvantage of similarity metrics
* need hand-engineered statistics
* features cannot be learned
* the node representations are given

## Objectives
* learn the node and edge representations

<img src="images/orig_embedding_hamilton.png" width="800"/>
ENC(u) transforms a node to an embedding. Any type of network can be used, but usually some form of GNN. 
* Shallow encoders only consider graph topology through metrics
* GNN can also consider node and edge features
* GNN can balance the effect of topology and that of features obtained via social networks for example. 

## Standard decoder
* encode all the does to embeddings $Z = {z(u)}$
* decode the embeddings to reconstruct the neighborhood. 

<img src="images/standard_decoder_hamilton.png" width="800"/>

## Pairwise decoders
$
\hspace{2in} DEC:  \Re{d} \times \Re{d} ==>  \Rez^+
$

Interpretation: output of decoder measures the similarity between two nodes. 

## Supervised learning
Given a collection of edges, compute a similarity measure $S(u,v)$. The objective is 
for the decoder to *reconstruct* $S(u,v)$.

## Loss function
$
\hspace{2in}\cal{L} = \sum_{(u,v)\in\cal{D}} l(\rm{DEC}(z_u, z_v), S(u,v))
$

<img src="images/similarity_table_hamilton.png" width=800/>

Different choices of decoder, similrity measure and loss function lead to different approaches. No 
approach is best for all tasks. 

Please read Hamilton Book, chapters 2 and 3 to review the above material.

# Analyze code written to use GraphSage to solve a link prediction problem
Code and notes are taken from the blog of [ Samar Khanna, Sarthak Consul, and Tanish Jain, Stanford students](https://medium.com/stanford-cs224w/online-link-prediction-with-graph-neural-networks-46c1054f2aa4), which describes how to use GraphSage for link prediction.

## Colab notebook
* https://colab.research.google.com/drive/1nb-BhjggQq4E7Ew9RxlydUbXF7Tpk6mH

## Github repository
* https://github.com/samar-khanna/cs224w-project

We will analyze the structure of the code, and run it next week. 

* The code uses [pyTorch_geometric](https://pytorch-geometric.readthedocs.io/en/latest/), and OGB ([Open Graph Benchmark](https://ogb.stanford.edu))
It is important to analyze source code and learn as you code. 

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric as pyg
from torch_geometric.data import DataLoader
from torch_geometric.utils import negative_sampling
from ogb.linkproppred import PygLinkPropPredDataset, Evaluator #needed to extract and evaluate the ogb-ddi dataset
import matplotlib.pyplot as plt #needed to visualize loss curves
import numpy as np

ModuleNotFoundError: No module named 'torch_geometric'

## GraphSage model definition
Found in the [PyGeometric Docs](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html?highlight=nn.sageconv#torch_geometric.nn.conv.SAGEConv). 

In [10]:
class GNNStack(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout, emb=False):
        super(GNNStack, self).__init__()
        
        # Model obtained from pyg
        # nn.SAGEConv probably becomes nn.conv.SAGEConv
        conv_model = pyg.nn.SAGEConv

        self.convs = nn.ModuleList()
        self.convs.append(conv_model(input_dim, hidden_dim))
        self.dropout = dropout
        self.num_layers = num_layers
        self.emb = emb

        # Create num_layers GraphSAGE convs
        assert (self.num_layers >= 1), 'Number of layers is not >=1'
        for l in range(self.num_layers - 1):
            self.convs.append(conv_model(hidden_dim, hidden_dim))

        # post-message-passing processing 
        self.post_mp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim), nn.Dropout(self.dropout),
            nn.Linear(hidden_dim, output_dim))

    def forward(self, x, edge_index):
        # x is the initial embedding
        # In this code, it is the weight from an embedding layer. 
        # If there were node attributes, x would be the initial node attributes
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.post_mp(x)

        # Return final layer of embeddings if specified
        if self.emb:
            return x

        # Else return class probabilities
        return F.log_softmax(x, dim=1)

    def loss(self, pred, label):
        return F.nll_loss(pred, label)

* This model has multiple layers, an ReLU activation function, uses dropout
* The forward function returns either embeddings, or a softmax (if a probability is required)
* Input to the forward function takes x (list of nodes) and edge_index: 

## Decoder definition

In [8]:
class LinkPredictor(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(LinkPredictor, self).__init__()

        # Create linear layers
        self.lins = nn.ModuleList()
        self.lins.append(nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        # x_i and x_j are both of shape (E, D)
        x = x_i * x_j
        for lin in self.lins[:-1]:
            x = lin(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.sigmoid(x)

* Input to the *forward* method is two nodes. 
* x is the pointwise multiplication between the embeddings of nodes i and j. 
* The link predictor returns a probablity. 
    * assume a link if probability > 0.5
    * no link if probability < 0.5 .

## Training
Source code to [negative_sampling](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/utils/negative_sampling.html) in PyGeometric.<br/> 
The *negative_sampling* method only considers true negative samples. 

In [12]:
def train(model, link_predictor, emb, edge_index, pos_train_edge, batch_size, optimizer):
    """
    Runs offline training for model, link_predictor and node embeddings given the message
    edges and supervision edges.
    :param model: Torch Graph model used for updating node embeddings based on message passing
    :param link_predictor: Torch model used for predicting whether edge exists or not
    :param emb: (N, d) Initial node embeddings for all N nodes in graph
    :param edge_index: (2, E) Edge index for all edges in the graph
    :param pos_train_edge: (PE, 2) Positive edges used for training supervision loss
    :param batch_size: Number of positive (and negative) supervision edges to sample per batch
    :param optimizer: Torch Optimizer to update model parameters
    :return: Average supervision loss over all positive (and correspondingly sampled negative) edges
    """
    model.train()
    link_predictor.train()

    train_losses = []

    for edge_id in DataLoader(range(pos_train_edge.shape[0]), batch_size, shuffle=True):
        optimizer.zero_grad()

        # Run message passing on the inital node embeddings to get updated embeddings
        # emb: initial node embedding <<<<<
        # edge_index: list of edges (2,E)
        node_emb = model(emb, edge_index)  # (N, d)

        # Predict the class probabilities on the batch of positive edges using link_predictor
        pos_edge = pos_train_edge[edge_id].T  # (2, B)
        pos_pred = link_predictor(node_emb[pos_edge[0]], node_emb[pos_edge[1]])  # (B, )

        # Sample negative edges (same number as number of positive edges) and predict class probabilities 
        # negative_sampling is built into torch_geometric
        neg_edge = negative_sampling(edge_index, num_nodes=emb.shape[0],
                                     num_neg_samples=edge_id.shape[0], method='dense')  # (Ne,2)
        neg_pred = link_predictor(node_emb[neg_edge[0]], node_emb[neg_edge[1]])  # (Ne,)

        # Compute the corresponding negative log likelihood loss on the positive and negative edges
        loss = -torch.log(pos_pred + 1e-15).mean() - torch.log(1 - neg_pred + 1e-15).mean()

        # Backpropagate and update parameters
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
    return sum(train_losses) / len(train_losses)

## Testing
Use the Hits@K metric ([more info](https://stackoverflow.com/questions/58796367/how-is-hitsk-calculated-and-what-does-it-mean-in-the-context-of-link-prediction)).

In [13]:
def test(model, predictor, emb, edge_index, split_edge, batch_size, evaluator):
    """
    Evaluates graph model on validation and test edges
    :param model: Torch Graph model used for updating node embeddings based on message passing
    :param predictor: Torch model used for predicting whether edge exists or not
    :param emb: (N, d) Initial node embeddings for all N nodes in graph
    :param edge_index: (2, E) Edge index for all edges in the graph
    :param split_edge: Dictionary of (e, 2) edges for val pos/neg and test pos/neg edges
    :param batch_size: Number of positive (and negative) supervision edges to sample per batch
    :param evaluator: OGB evaluator to calculate hits @ k metric
    :return: hits @ k results
    """
    model.eval()
    predictor.eval()

    node_emb = model(emb, edge_index)

    pos_valid_edge = split_edge['valid']['edge'].to(emb.device)
    neg_valid_edge = split_edge['valid']['edge_neg'].to(emb.device)
    pos_test_edge = split_edge['test']['edge'].to(emb.device)
    neg_test_edge = split_edge['test']['edge_neg'].to(emb.device)

    pos_valid_preds = []
    for perm in DataLoader(range(pos_valid_edge.size(0)), batch_size):
        edge = pos_valid_edge[perm].t()
        pos_valid_preds += [predictor(node_emb[edge[0]], node_emb[edge[1]]).squeeze().cpu()]
    pos_valid_pred = torch.cat(pos_valid_preds, dim=0)

    neg_valid_preds = []
    for perm in DataLoader(range(neg_valid_edge.size(0)), batch_size):
        edge = neg_valid_edge[perm].t()
        neg_valid_preds += [predictor(node_emb[edge[0]], node_emb[edge[1]]).squeeze().cpu()]
    neg_valid_pred = torch.cat(neg_valid_preds, dim=0)

    pos_test_preds = []
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size):
        edge = pos_test_edge[perm].t()
        pos_test_preds += [predictor(node_emb[edge[0]], node_emb[edge[1]]).squeeze().cpu()]
    pos_test_pred = torch.cat(pos_test_preds, dim=0)

    neg_test_preds = []
    for perm in DataLoader(range(neg_test_edge.size(0)), batch_size):
        edge = neg_test_edge[perm].t()
        neg_test_preds += [predictor(node_emb[edge[0]], node_emb[edge[1]]).squeeze().cpu()]
    neg_test_pred = torch.cat(neg_test_preds, dim=0)

    results = {}
    for K in [20, 50, 100]:
        evaluator.K = K
        valid_hits = evaluator.eval({
            'y_pred_pos': pos_valid_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        test_hits = evaluator.eval({
            'y_pred_pos': pos_test_pred,
            'y_pred_neg': neg_test_pred,
        })[f'hits@{K}']

        results[f'Hits@{K}'] = (valid_hits, test_hits)

    return results

## Load the dataset
* The [link prediction dataset](https://ogb.stanford.edu/docs/linkprop/) can be downloaded directly from within OGB. 

dataset = PygLinkPropPredDataset(name="ogbl-ddi", root='./dataset/') #download the dataset

In [15]:
## Hyperparameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optim_wd = 0
epochs = 300
hidden_dim = 256
dropout = 0.3
num_layers = 2
lr = 3e-3
node_emb_dim = 256
batch_size = 64 * 1024

## Run the code
* Note that initial node embeddings are specified via an Embeddings layer, i.e., a weight matrix. 
* More genreally, the initial embeddings would be the node features (none in this case). 

In [16]:
split_edge = dataset.get_edge_split()
pos_train_edge = split_edge['train']['edge'].to(device)

graph = dataset[0]
edge_index = graph.edge_index.to(device)

evaluator = Evaluator(name='ogbl-ddi')

# The nodes have no features. The initial embeddings are random numbers. 
#  GE: More generally, the initial embeddings would be node features. 
emb = torch.nn.Embedding(graph.num_nodes, node_emb_dim).to(device) # each node has an embedding that has to be learnt

# Define the Encoder
model = GNNStack(node_emb_dim, hidden_dim, hidden_dim, num_layers, dropout, emb=True).to(device) # the graph neural network that takes all the node embeddings as inputs to message pass and agregate

# Define the Decoder
link_predictor = LinkPredictor(hidden_dim, hidden_dim, 1, num_layers + 1, dropout).to(device) # the MLP that takes embeddings of a pair of nodes and predicts the existence of an edge between them

# Note the parameters: model, Link, embedding matrix
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(link_predictor.parameters()) + list(emb.parameters()),
    lr=lr, weight_decay=optim_wd
)

train_loss = []
val_hits = []
test_hits = []
for e in range(epochs):
    # GE: notice the input emb.weight. How is it used? 
    loss = train(model, link_predictor, emb.weight, edge_index, pos_train_edge, batch_size, optimizer)
    print(f"Epoch {e + 1}: loss: {round(loss, 5)}")
    train_loss.append(loss)

    if (e+1)%10 ==0:
        result = test(model, link_predictor, emb.weight, edge_index, split_edge, batch_size, evaluator)
        val_hits.append(result['Hits@20'][0])
        test_hits.append(result['Hits@20'][1])
        print(result)

plt.title('Link Prediction on OGB-ddi using GraphSAGE GNN')
plt.plot(train_loss,label="training loss")
plt.plot(np.arange(9,epochs,10),val_hits,label="Hits@20 on validation")
plt.plot(np.arange(9,epochs,10),test_hits,label="Hits@20 on test")
plt.xlabel('Epochs')
plt.legend()
plt.show()

NameError: name 'dataset' is not defined

# Online Link Predictor
* For next week. 